In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import torchvision

In [9]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


In [10]:
# Train and test dataset 
train = torchvision.datasets.MNIST()
test = torchvision.datasets.MNIST()

TypeError: MNIST.__init__() missing 1 required positional argument: 'root'

In [11]:
# dataloader

In [12]:
# VIT
# Paper https://arxiv.org/abs/2010.11929
class ViT(torch.nn.Module):
    def __init__(self, P, N, D, H, MLP):
        super(ViT, self).__init__()
        self.D = D
        self.N = N
        self.rand = torch.normal(0,1, size=(1, 1, D))
        self.location_encoding = torch.tensor([[i for i in range(0,17)]])
        self.position_embeddings = nn.Embedding(N+1, D)
        self.flatten = nn.Flatten(2)
        self.patch_projection = nn.Linear(P*P, D)
        self.encoder_1 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(D, H, 3000, activation="gelu"),
            num_layers=4
        )
        self.mlp_1 = nn.Linear(self.D, 250)
        self.mlp_2 = nn.Linear(250, 100)
        self.mlp_3 = nn.Linear(100, 10)
        self.amax = torch.argmax
    
    def forward(self, x):
        rand = self.rand.repeat(x.shape[0], 1, 1)
        x_loc = self.position_embeddings(self.location_encoding)
        x = self.flatten(x)
        x_p = self.patch_projection(x)
        encoding = torch.cat([rand, x_p,], dim=1) + x_loc
        encoding = F.noramlize(encoding)
        z = self.encoder_1(encoding)
        head = z[:, 0]
        x = self.mlp_1(head)
        x = self.mlp_2(x)
        x = F.tanh(x)
        x = self.mlp_3(x)
        output = F.log_softmax(x, dim=1)
        return output